## Script copy

In [1]:
from typing import List, Dict, Tuple
import numpy as np
from tqdm import trange
from tqdm import tqdm
from functools import reduce
from operator import mul

class Case:
    def __init__(self, arguments:Dict[str, int]):
        self.arguments = arguments
    
    def __str__(self):
        return str(self.arguments)
    
    def size(self):
        return reduce(mul, self.arguments.values())

class Parameters:
    def __init__(self, e_cu, e_cu_X, q_ic, rp_c, b, k, l_c, m_i, n_i, m_i_X, n_i_X, t_hd):
        self.e_cu = e_cu
        self.e_cu_X = e_cu_X
        self.q_ic = q_ic
        self.rp_c = rp_c
        self.b = b
        self.k = k
        self.l_c = l_c
        self.m_i = m_i
        self.n_i = n_i
        self.m_i_X = m_i_X
        self.n_i_X = n_i_X
        self.t_hd = t_hd

class SolutionResult:
    def __init__(self, case: Case, value: float, duration:int):
        self.case = case
        self.value = value
        self.duration = duration
    def __str__(self):
        return f"<case: {self.case}, value: {self.value}, duration: {self.duration}>"


class Experiment:
    def __init__(self, cases: List[Case]):
        self.cases = cases
    
    def run_cases_with(self, solution) -> List[Tuple[Case, SolutionResult]]:
        return [solution.run(case) for case in tqdm(self.cases, f"Case ->")]

class Solution:
    c_i = 0
    u_i = 1
    h_i = 2
    d_i = 3
    def __init__(self, name: str):
        self.name = name

    def run(self, case:Case)->SolutionResult:
        #TODO solve with a solution algorith
        parameters = self.generate_parameters(case)
        print(parameters)
        duration = 10
        value = 2.1
        return SolutionResult(case, value, duration)

    def generate_parameters(self, case: Case) -> Parameters:
        import numpy as np
        np.random.seed(23)
        C = case.arguments["C"] # number of campaigns
        U = case.arguments["U"]  # number of customers.
        H = case.arguments["H"]  # number of channels.
        D = case.arguments["D"]  # number of planning days.
        I = case.arguments["I"]  # number of quota categories.
        P = case.arguments["P"]  # number of priority categories.
        ##eligibility
        e_cu = np.random.choice(2,(C, U)) #e_cu = np.ones((C, U), dtype='int8')
        ##quota categories
        q_ic = np.random.choice(2, (I,C)) #q_ic = np.zeros((I,C), dtype='int8')
        ##priority categories
        r_p = np.random.choice(100, P) #r_p = np.ones(P, dtype='int8')
        rp_c = np.array([r_p[r] for r in np.random.choice(P, C)])
        ##blokage
        b = 7
        ##daily blokage
        k = 3
        ##campaign blockage
        l_c = np.random.choice([2,3,4],C)
        ##quota limitations daily/weekly
        m_i = np.random.choice([4,3,5],I)#m_i = np.ones((I), dtype='int8')*10
        n_i = np.random.choice([1,3,2],I)#n_i = np.ones((I), dtype='int8')*10
        ##capacity for channel
        t_hd = np.random.choice([U*.7, U*.6, U*.5], (H, D))
        e_cu_X = np.stack([np.stack([e_cu for _ in range(H)], axis=2) for _ in range(D)], axis=3)
        m_i_X = np.stack([m_i for _ in range(U)], axis=1)
        n_i_X = np.stack([n_i for _ in range(U)], axis=1)
        return Parameters(e_cu,e_cu_X,q_ic,rp_c,b,k,l_c,m_i, n_i, m_i_X, n_i_X, t_hd)

#Single Constraint Functions
    def eligibility(self, e_cu, X, c, u, h, d):
        return X[c,u,h,d]<=e_cu[c,u]
    def weekly_limitation(self, b, X, u):
        return X[:,u,:,:].sum() <= b
    def daily_limitation (self, k, X, u, d):
        return X[:,u,:,d].sum() <= k
    def campaign_limitation(self, l_c, X, c, u):
        return X[c,u,:,:].sum() <= l_c[c]
    def weekly_quota(self, m_i, q_ic, X, u):
        return all((q_ic * X[:,u,:,:].sum(axis=(1,2))).sum(axis=1)<=m_i)
    def daily_quota(self, n_i, q_ic, X, u, d):
        return all((q_ic * X[:,u,:,d].sum(axis=(1))).sum(axis=1)<=n_i)
    def channel_capacity(self, t_hd, X, h, d):
        return X[:,:,h,d].sum() <= t_hd[h,d]

    def check(self, X, PMS, indicies):
        if not self.eligibility(PMS.e_cu, X, indicies[self.c_i],indicies[self.u_i],indicies[self.h_i],indicies[self.d_i]):
            return False
        if not self.weekly_limitation(PMS.b, X, indicies[self.u_i]):
            return False
        if not self.daily_limitation(PMS.k, X, indicies[self.u_i],indicies[self.d_i]):
            return False
        if not self.campaign_limitation(PMS.l_c, X, indicies[self.c_i],indicies[self.u_i]):
            return False
        if not self.weekly_quota(PMS.m_i, PMS.q_ic, X, indicies[self.u_i]):
            return False
        if not self.daily_quota(PMS.n_i, PMS.q_ic, X, indicies[self.u_i],indicies[self.d_i]):
            return False
        if not self.channel_capacity(PMS.t_hd, X, indicies[self.h_i],indicies[self.d_i]):
            return False
        return True
    
    def objective_fn(self, rp_c, X):
        return np.matmul(rp_c, X.sum(axis=(1,2,3)))

#Bulk Constraint Functions
    def X_eligibility (self, e_cu_X, X):
        return (X <= e_cu_X).all()
    def X_weekly_limitation (self, b, X):
        return (X.sum(axis=(0,2,3))<=b).all()
    def X_daily_limitation (self, k, X):
        return (X.sum(axis=(0)).sum(axis=(1))<=k).all()
    def X_campaign_limitation (self, l_c, X):
        return np.all(X.sum(axis=(2,3)).T<=l_c)
    def X_weekly_quota (self, m_i, q_ic, X):
        q_range = range(q_ic.shape[0])
        for i in q_range:
            quota_i = np.all(X.sum(axis=(2,3)).T * q_ic[i, ].T  <= m_i[i])
            if not quota_i:
                return False
        return True
    def X_channel_capacity (self, t_hd, X):
        return np.all(X.sum(axis=(0,1))<=t_hd)

    def X_daily_quota (self, n_i, q_ic, X):
        q_range = range(q_ic.shape[0])
        for i in q_range:
            quota_i = np.all((q_ic[i,].T * X.sum(axis=2).T).sum(2) <= n_i[i])
            if not quota_i:
                return False
        return True

    def X_check(self, PMS, X):
        if not self.X_eligibility(PMS.e_cu_X, X):
            return False
        if not self.X_weekly_limitation(PMS.b, X):
            return False
        if not self.X_daily_limitation(PMS.k, X):
            return False
        if not self.X_campaign_limitation(PMS.l_c, X):
            return False
        if not self.X_weekly_quota(PMS.m_i, PMS.q_ic, X):
            return False
        if not self.X_daily_quota(PMS.n_i, PMS.q_ic, X):
            return False
        if not self.X_channel_capacity(PMS.t_hd, X):
            return False
        return True

In [2]:
#from src.experiment import Solution,  SolutionResult, Case, Experiment, Parameters
import numpy as np

In [3]:
class LpRelSolution(Solution):
    def __init__(self):
        super().__init__("LP-Relaxation")

    def round_with_greedy(self, X_non_integral, X, PMS, D, H, U):
        from tqdm import tqdm
        from tqdm import trange
        for c in np.argsort(PMS.rp_c):#tqdm(np.argsort(PMS.rp_c), desc="Campaigns Loop"):
            for d in range(D):#trange(D, desc=f"Days Loop for campaign-{c}"):
                for h in range(H):
                    for u in range(U):
                        if (c,u,h,d) in X_non_integral and X[c,u,h,d]==1 and not self.check(X, PMS, (c, u, h, d)):
                            X[c,u,h,d]=0

    def run(self, case:Case)->SolutionResult:
        from time import time
        from docplex.mp.model import Model
        start_time = time()
        #seed randomization
        C = case.arguments["C"] # number of campaigns
        U = case.arguments["U"]  # number of customers.
        H = case.arguments["H"]  # number of channels.
        D = case.arguments["D"]  # number of planning days.
        I = case.arguments["I"]  # number of quota categories.
        P = case.arguments["P"]  # number of priority categories.
        PMS:Parameters = super().generate_parameters(case)
        mdl = Model(name='Campaign Optimization')
        #variables
        X_cuhd = {(c,u,h,d): mdl.continuous_var(lb=0, ub=1, name=f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,C)
            for u in range(0,U) 
            for h in range(0,H)
            for d in range(0,D)}
        #objectivefunction
        maximize = mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * PMS.rp_c[c]
                  for c in range(0,C)
                  for u in range(0,U) 
                  for h in range(0,H) 
                  for d in range(0,D)]))
        #constraints
        eligibilitiy = mdl.add_constraints(
            (X_cuhd[(c,u,h,d)] <= PMS.e_cu[c,u]
            for c in range(0,C)
            for u in range(0,U) 
            for h in range(0,H) 
            for d in range(0,D)))
        
        weekly_communication = mdl.add_constraints((
            (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,D) 
               for c in range(0,C) 
               for h in range(0,H)) <= PMS.b)
            for u in range(0,U)))

        daily_communication = mdl.add_constraints((
            (mdl.sum(X_cuhd[(c,u,h,d)]  
                    for c in range(0,C) 
                    for h in range(0,H)) <= PMS.k)
                for d in range(0,D)
                for u in range(0,U)))
        
        campaign_communication = mdl.add_constraints((
            (mdl.sum(X_cuhd[(c,u,h,d)]  
                for h in range(0,H) 
                for d in range(0,D)) <= PMS.l_c[c] )
            for c in range(0,C)
            for u in range(0,U)))

        weekly_quota = mdl.add_constraints((
            (mdl.sum(X_cuhd[(c,u,h,d)]*PMS.q_ic[i,c]
                for c in range(0,C)
                for h in range(0,H) 
                for d in range(0,D)) <= PMS.m_i[i])
            for u in range(0,U)
            for i in range(0,I)))
        
        result = mdl.solve(log_output=False)
        v_non_integral = result.objective_value
        X_non_integral = [tuple(int(i.split(":")[1]) for i in ky.split("_")[1:]) for ky,val in result.as_name_dict().items() if val!=1]
        X_cuhd2 = np.zeros((C,U,H,D), dtype='int')
        for ky,_ in result.as_name_dict().items():
            exec(f'X_cuhd2{[int(i.split(":")[1]) for i in ky.split("_")[1:]]} = 1', {}, {'X_cuhd2':X_cuhd2})
        v_non_opt = self.objective_fn(PMS.rp_c, X_cuhd2)
        self.round_with_greedy(X_non_integral, X_cuhd2, PMS, D, H, U)
        if(v_non_opt!=v_non_integral):
            print(f"Non-optimistic & integral solution: {v_non_opt}")
            print(f"Non-integral solution: {v_non_integral}")
            print(f"Optimistic solution: {self.objective_fn(PMS.rp_c, X_cuhd2)}")
        value = self.objective_fn(PMS.rp_c, X_cuhd2)
        end_time = time()
        duration = end_time - start_time
        return SolutionResult(case, value, round(duration,4))

In [4]:
if __name__ == '__main__':
    cases = [
            Case({"C":20,"U":500,"H":3, "D":7, "I":3, "P":3})
            ]
    expr = Experiment(cases)
    solutions = expr.run_cases_with(LpRelSolution())
    for solution in solutions:
        print(solution)

#
#<case: {'C': 5, 'U': 100, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 12180, duration: 1.5658>
#<case: {'C': 10, 'U': 1000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 341667, duration: 29.2164>
#<case: {'C': 15, 'U': 10000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 4836874, duration: 445.1445>

#<case: {'C': 2, 'U': 100, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 13083, duration: 0.8805>
#<case: {'C': 5, 'U': 100, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 12180, duration: 1.5107>
#<case: {'C': 5, 'U': 200, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 63096, duration: 2.7442>
#<case: {'C': 5, 'U': 1000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 74664, duration: 14.9263>
#<case: {'C': 10, 'U': 1000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 341667, duration: 28.4282>
#<case: {'C': 10, 'U': 2000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 697860, duration: 109.7368>
#<case: {'C': 10, 'U': 3000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 1672442, duration: 92.7799>
#<case: {'C': 10, 'U': 4000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 1161046, duration: 122.5116>
#<case: {'C': 10, 'U': 5000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 1982340, duration: 148.8915>
#<case: {'C': 20, 'U': 10000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 5506153, duration: 632.4157>
#<case: {'C': 20, 'U': 20000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 4945718, duration: 1440.8951>
#<case: {'C': 20, 'U': 30000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 12309790, duration: 4512.4216>
#<case: {'C': 20, 'U': 40000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 26450884, duration: 4919.4901>
#<case: {'C': 20, 'U': 50000, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 22563024, duration: 8466.245>

Case ->: 100%|██████████| 1/1 [00:41<00:00, 41.16s/it]

<case: {'C': 20, 'U': 500, 'H': 3, 'D': 7, 'I': 3, 'P': 3}, value: 252556, duration: 40.0028>


In [15]:
del solution